# Practica 3
## Magdaleno Flores Nilton Sebastian

### 1.Preprocesamiento
### 2.Análisis de Datos

#### Importación de librerías y carga de datos

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

In [11]:
path_ds = 'CTG.csv'
df = pd.read_csv(path_ds)

In [12]:
df.head()

,FileName,Date,SegFile,b,e,LBE,LB,AC,FM,UC,ASTV,MSTV,ALTV,MLTV,DL,DS,DP,DR,Width,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,A,B,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
0,Variab10.txt,12/1/1996,CTG0001.txt,240.0,357.0,120.0,120.0,0.0,0.0,0.0,73.0,0.5,43.0,2.4,0.0,0.0,0.0,0.0,64.0,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,2.0
1,Fmcs_1.txt,5/3/1996,CTG0002.txt,5.0,632.0,132.0,132.0,4.0,0.0,4.0,17.0,2.1,0.0,10.4,2.0,0.0,0.0,0.0,130.0,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
2,Fmcs_1.txt,5/3/1996,CTG0003.txt,177.0,779.0,133.0,133.0,2.0,0.0,5.0,16.0,2.1,0.0,13.4,2.0,0.0,0.0,0.0,130.0,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
3,Fmcs_1.txt,5/3/1996,CTG0004.txt,411.0,1192.0,134.0,134.0,2.0,0.0,6.0,16.0,2.4,0.0,23.0,2.0,0.0,0.0,0.0,117.0,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
4,Fmcs_1.txt,5/3/1996,CTG0005.txt,533.0,1147.0,132.0,132.0,4.0,0.0,5.0,16.0,2.4,0.0,19.9,0.0,0.0,0.0,0.0,117.0,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 40 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FileName  2126 non-null   object 
 1   Date      2126 non-null   object 
 2   SegFile   2126 non-null   object 
 3   b         2126 non-null   float64
 4   e         2126 non-null   float64
 5   LBE       2126 non-null   float64
 6   LB        2126 non-null   float64
 7   AC        2126 non-null   float64
 8   FM        2127 non-null   float64
 9   UC        2127 non-null   float64
 10  ASTV      2127 non-null   float64
 11  MSTV      2127 non-null   float64
 12  ALTV      2127 non-null   float64
 13  MLTV      2127 non-null   float64
 14  DL        2128 non-null   float64
 15  DS        2128 non-null   float64
 16  DP        2128 non-null   float64
 17  DR        2128 non-null   float64
 18  Width     2126 non-null   float64
 19  Min       2126 non-null   float64
 20  Max       2126 non-null   floa

#### Creación de la función para determinar los nulos y eliminar las columnas con 20% o más de nulos

In [ ]:
def check_data_completeness_Magdaleno_Flores_Nilton_Sebastian(df):
    """
    Función general para verificar la completitud y calidad de los datos.
    Parámetros:
    df (pd.DataFrame): DataFrame a analizar.
    Retorna:
    pd.DataFrame: DataFrame con el análisis de completitud y calidad de cada columna.
    """
    res = []                                                        # Lista para almacenar resultados
    for col in df.columns:
        dtype = df[col].dtype                                       # Tipo de dato
        total_rows = len(df)                                        # Total de filas
        null_count = df[col].isnull().sum()                         # Conteo de valores nulos
        null_pct = (null_count / total_rows) * 100                  # Porcentaje de nulos
        completeness_pct = 100 - null_pct                           # Porcentaje de completitud        
        unique_vals = df[col].nunique()                             # Valores únicos
        
        # Estadísticos de dispersión y clasificación
        if pd.api.types.is_numeric_dtype(df[col]):
            std_dev = df[col].std()                                 # Desviación estándar
            variance = df[col].var()                                # Varianza
            # Clasificación automática
            if unique_vals > 10:
                col_type = 'Continua'
            else:
                col_type = 'Discreta'
        else:
            std_dev = np.nan                                        # Desviación estándar no aplicable
            variance = np.nan                                       # Varianza no aplicable
            col_type = 'Categórica'
            if unique_vals < 10:
                 col_type = 'Discreta (Cat)'

        res.append({
            'Columna': col,
            'Nulos': null_count,
            '% Completitud': round(completeness_pct, 2),
            'Tipo Dato': dtype,
            'Valores Únicos': unique_vals,
            'Std Dev': round(std_dev, 2) if not pd.isna(std_dev) else np.nan,
            'Varianza': round(variance, 2) if not pd.isna(variance) else np.nan,
            'Clasificación': col_type
        })
    
    return pd.DataFrame(res)

In [ ]:
# Ejecutar análisis inicial
analysis_df = check_data_completeness_Magdaleno_Flores_Nilton_Sebastian(df)
analysis_df

#### Imputación de valores y Detección de valores atípicos y outliers

In [ ]:
def process_data(df_input):
    df_proc = df_input.copy()
    
    # 1. Eliminar columnas con más del 20% de valores nulos
    null_pct = df_proc.isnull().mean()
    cols_to_drop = null_pct[null_pct > 0.2].index
    if len(cols_to_drop) > 0:
        print(f"Columnas eliminadas (>20% nulos): {list(cols_to_drop)}")
        df_proc = df_proc.drop(columns=cols_to_drop)
    else:
        print("No hay columnas con >20% de nulos para eliminar.")
    
    # 2. Imputar valores faltantes restantes
    numeric_cols = df_proc.select_dtypes(include=[np.number]).columns
    categorical_cols = df_proc.select_dtypes(exclude=[np.number]).columns
    
    # Imputación Numérica: Usamos Mediana por ser robusta a outliers
    for col in numeric_cols:
        if df_proc[col].isnull().sum() > 0:
            median_val = df_proc[col].median()
            df_proc[col] = df_proc[col].fillna(median_val)
            
    # Imputación Categórica: Moda
    for col in categorical_cols:
        if df_proc[col].isnull().sum() > 0:
            mode_val = df_proc[col].mode()[0]
            df_proc[col] = df_proc[col].fillna(mode_val)
            
    # 3. Detectar y tratar valores atípicos (Outliers) con IQR
    # Excluimos columnas target o identificadores si es necesario
    # En CTG, 'NSP' y 'CLASS' son targets, no deberíamos clipearlas.
    cols_exclude_outlier = ['NSP', 'CLASS', 'Tendency']
    cols_for_outliers = [c for c in numeric_cols if c not in cols_exclude_outlier]
    
    for col in cols_for_outliers:
        Q1 = df_proc[col].quantile(0.25)
        Q3 = df_proc[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Tratamiento: Clipping (limitar valores al rango)
        # Esto preserva la distribución mejor que eliminar filas
        df_proc[col] = np.clip(df_proc[col], lower_bound, upper_bound)
        
    return df_proc

df_clean = process_data(df)

# Verificar que no queden nulos
print(f"\nTotal de nulos después del procesamiento: {df_clean.isnull().sum().sum()}")